# Projeto Web Scraping ETF

## Instalando bibliotecas necessárias

In [1]:
!pip install webdriver-manager
!pip install selenium
!pip install html5lib

## Importar Bibliotecas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

## Acessando Navegador

In [3]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

## Acessar elementos do HTML

In [4]:
time.sleep(5)
botao_100 = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')
driver.execute_script("arguments[0].click();", botao_100)

## Pegando número de página da tabela

In [5]:
numero_paginas = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')
numero_paginas = numero_paginas.text.replace("of ", "")
numero_paginas = int(numero_paginas)
print(numero_paginas)

32


## Lendo tabela de dados

In [6]:
lista_de_tabelas_por_pagina = []

for pagina in range(0, numero_paginas):
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')
    html_tabela = tabela.get_attribute("outerHTML")
    tabela_final = pd.read_html(html_tabela)[0]
    lista_de_tabelas_por_pagina.append(tabela_final)
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)

base_de_dados_completa = pd.concat(lista_de_tabelas_por_pagina)
display(base_de_dados_completa)

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B
2,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B
3,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B
...,...,...,...,...,...,...
8,CVSB,Calvert Ultra-Short Investment Grade ETF,"Fixed Income: Global - Broad Market, Broad-bas...",Morgan Stanley,0.24%,--
9,CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--
10,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--
11,CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,--,--


## Ler dados de rentabilidade

### Mudando de aba

In [7]:
botao_aba = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span''')
driver.execute_script("arguments[0].click();", botao_aba)

### Voltando a primeira página

In [8]:
for pagina in range(0, numero_paginas):
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)

### Criando tabela de performanace

In [9]:
lista_de_tabelas_por_pagina = []

for pagina in range(0, numero_paginas):
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table''')
    html_tabela = tabela.get_attribute("outerHTML")
    tabela_final = pd.read_html(html_tabela)[0]
    lista_de_tabelas_por_pagina.append(tabela_final)
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)

base_de_dados_performance = pd.concat(lista_de_tabelas_por_pagina)
display(base_de_dados_performance)

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,4.96%,9.32%,6.45%,-9.58%,8.73%,11.15%,12.44%,02/09/23
1,IVV,iShares Core S&P 500 ETF,5.01%,9.33%,6.47%,-9.55%,8.74%,11.17%,12.50%,02/09/23
2,VTI,Vanguard Total Stock Market ETF,5.42%,9.66%,7.03%,-10.06%,8.29%,10.66%,12.08%,02/09/23
3,VOO,Vanguard S&P 500 ETF,4.99%,9.31%,6.48%,-9.57%,8.71%,11.17%,12.50%,02/09/23
4,QQQ,Invesco QQQ Trust,11.51%,14.85%,13.29%,-17.13%,10.27%,14.91%,17.15%,02/09/23
...,...,...,...,...,...,...,...,...,...,...
8,CVSB,Calvert Ultra-Short Investment Grade ETF,--,--,--,--,--,--,--,02/09/23
9,CVLC,Calvert US Large-Cap Core Responsible Index ETF,--,--,--,--,--,--,--,02/09/23
10,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,--,--,--,--,--,--,--,02/09/23
11,CVIE,Calvert International Responsible Index ETF,--,--,--,--,--,--,--,02/09/23


## Construindo tabela final

In [10]:
base_de_dados_completa = base_de_dados_completa.set_index("Ticker")
base_de_dados_completa

,Name,Segment,Issuer,Expense Ratio,AUM
Ticker,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B
...,...,...,...,...,...
CVSB,Calvert Ultra-Short Investment Grade ETF,"Fixed Income: Global - Broad Market, Broad-bas...",Morgan Stanley,0.24%,--
CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--


In [11]:
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")
base_de_dados_performance = base_de_dados_performance[['1 Year', '5 Years', '10 Years']]
base_de_dados_performance

,1 Year,5 Years,10 Years
Ticker,,,
SPY,-9.58%,11.15%,12.44%
IVV,-9.55%,11.17%,12.50%
VTI,-10.06%,10.66%,12.08%
VOO,-9.57%,11.17%,12.50%
QQQ,-17.13%,14.91%,17.15%
...,...,...,...
CVSB,--,--,--
CVLC,--,--,--
CVMC,--,--,--


In [12]:
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance)
base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B,-9.58%,11.15%,12.44%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B,-9.55%,11.17%,12.50%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B,-10.06%,10.66%,12.08%
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B,-9.57%,11.17%,12.50%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B,-17.13%,14.91%,17.15%
...,...,...,...,...,...,...,...,...
CVSB,Calvert Ultra-Short Investment Grade ETF,"Fixed Income: Global - Broad Market, Broad-bas...",Morgan Stanley,0.24%,--,--,--,--
CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--,--,--,--
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--,--,--,--


## Analisando tabela de retornos

### Removendo ETFS con informações faltantes

In [14]:
base_de_dados_performance = base_de_dados_performance.replace('--', pd.NA)
base_de_dados_performance = base_de_dados_performance.dropna()
base_de_dados_performance

,1 Year,5 Years,10 Years
Ticker,,,
SPY,-9.58%,11.15%,12.44%
IVV,-9.55%,11.17%,12.50%
VTI,-10.06%,10.66%,12.08%
VOO,-9.57%,11.17%,12.50%
QQQ,-17.13%,14.91%,17.15%
...,...,...,...
SZK,16.85%,-24.71%,-23.59%
LD,-3.70%,-5.28%,-2.39%
ERUS,-80.55%,-21.93%,-13.00%


### Transformas as rentabilidades em números decimais

In [15]:
base_de_dados_performance['1 Year'] = base_de_dados_performance['1 Year'].str.rstrip('%').astype(float)/100
base_de_dados_performance['5 Years'] = base_de_dados_performance['5 Years'].str.rstrip('%').astype(float)/100
base_de_dados_performance['10 Years'] = base_de_dados_performance['10 Years'].str.rstrip('%').astype(float)/100
base_de_dados_performance

,1 Year,5 Years,10 Years
Ticker,,,
SPY,-0.0958,0.1115,0.1244
IVV,-0.0955,0.1117,0.1250
VTI,-0.1006,0.1066,0.1208
VOO,-0.0957,0.1117,0.1250
QQQ,-0.1713,0.1491,0.1715
...,...,...,...
SZK,0.1685,-0.2471,-0.2359
LD,-0.0370,-0.0528,-0.0239
ERUS,-0.8055,-0.2193,-0.1300


### Incrementando tabela de dados de performance

In [17]:
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance, how = "inner")
base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B,-0.0958,0.1115,0.1244
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B,-0.0955,0.1117,0.1250
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B,-0.1006,0.1066,0.1208
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B,-0.0957,0.1117,0.1250
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B,-0.1713,0.1491,0.1715
...,...,...,...,...,...,...,...,...
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,0.1685,-0.2471,-0.2359
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0370,-0.0528,-0.0239
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,-0.8055,-0.2193,-0.1300


### Filtrar ETFS alavancados

In [20]:
base_de_dados_final = base_de_dados_final[~base_de_dados_final['Segment'].str.contains("Leveraged")]
base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B,-0.0958,0.1115,0.1244
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B,-0.0955,0.1117,0.1250
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B,-0.1006,0.1066,0.1208
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B,-0.0957,0.1117,0.1250
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B,-0.1713,0.1491,0.1715
...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.94M,-0.0285,-0.2685,-0.2854
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,0.1685,-0.2471,-0.2359
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0370,-0.0528,-0.0239


### Criar Ranking de rentabilidade por período

In [22]:
base_de_dados_final['rank_1_ano'] = base_de_dados_final['1 Year'].rank(ascending = False)
base_de_dados_final['rank_5_anos'] = base_de_dados_final['5 Years'].rank(ascending = False)
base_de_dados_final['rank_10_anos'] = base_de_dados_final['10 Years'].rank(ascending = False)
base_de_dados_final

C:\Users\Aloízio Borges\AppData\Local\Temp\ipykernel_12684\2276942365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_final['rank_1_ano'] = base_de_dados_final['1 Year'].rank(ascending = False)
C:\Users\Aloízio Borges\AppData\Local\Temp\ipykernel_12684\2276942365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_final['rank_5_anos'] = base_de_dados_final['5 Years'].rank(ascending = False)
C:\Users\Aloízio Borges\AppData\Local\Temp\ipykernel_12684\2276942365.py:3: SettingWit

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years,rank_1_ano,rank_5_anos,rank_10_anos
Ticker,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B,-0.0958,0.1115,0.1244,552.5,97.0,77.0
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B,-0.0955,0.1117,0.1250,549.0,93.5,74.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B,-0.1006,0.1066,0.1208,571.0,119.5,100.0
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B,-0.0957,0.1117,0.1250,551.0,93.5,74.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B,-0.1713,0.1491,0.1715,745.0,19.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.94M,-0.0285,-0.2685,-0.2854,273.0,808.0,816.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,0.1685,-0.2471,-0.2359,34.0,805.0,805.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0370,-0.0528,-0.0239,306.0,758.0,708.0


### Criando coluna de rank final

In [25]:
base_de_dados_final['rank_final'] = (base_de_dados_final['rank_1_ano'] + base_de_dados_final['rank_5_anos'] + base_de_dados_final['rank_10_anos'])
base_de_dados_final

C:\Users\Aloízio Borges\AppData\Local\Temp\ipykernel_12684\1027937134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_final['rank_final'] = (base_de_dados_final['rank_1_ano'] + base_de_dados_final['rank_5_anos'] + base_de_dados_final['rank_10_anos'])


,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years,rank_1_ano,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B,-0.0958,0.1115,0.1244,552.5,97.0,77.0,726.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B,-0.0955,0.1117,0.1250,549.0,93.5,74.5,717.0
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B,-0.1006,0.1066,0.1208,571.0,119.5,100.0,790.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B,-0.0957,0.1117,0.1250,551.0,93.5,74.5,719.0
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B,-0.1713,0.1491,0.1715,745.0,19.0,11.0,775.0
...,...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.94M,-0.0285,-0.2685,-0.2854,273.0,808.0,816.0,1897.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,0.1685,-0.2471,-0.2359,34.0,805.0,805.0,1644.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0370,-0.0528,-0.0239,306.0,758.0,708.0,1772.0


### Ordenando pela columa de rank final

In [27]:
melhores_rentabilidades_eft = base_de_dados_final.sort_values(by = "rank_final")
base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years,rank_1_ano,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B,-0.0958,0.1115,0.1244,552.5,97.0,77.0,726.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B,-0.0955,0.1117,0.1250,549.0,93.5,74.5,717.0
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B,-0.1006,0.1066,0.1208,571.0,119.5,100.0,790.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B,-0.0957,0.1117,0.1250,551.0,93.5,74.5,719.0
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B,-0.1713,0.1491,0.1715,745.0,19.0,11.0,775.0
...,...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.94M,-0.0285,-0.2685,-0.2854,273.0,808.0,816.0,1897.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,0.1685,-0.2471,-0.2359,34.0,805.0,805.0,1644.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0370,-0.0528,-0.0239,306.0,758.0,708.0,1772.0


In [28]:
melhores_rentabilidades_eft.head(10)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years,rank_1_ano,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.49B,0.1432,0.2809,0.1702,40.0,1.0,12.5,53.5
ICLN,iShares Global Clean Energy ETF,Equity: Global Renewable Energy,Blackrock,0.40%,$5.05B,0.0504,0.1895,0.1287,89.0,7.0,59.0,155.0
KBWP,Invesco KBW Property & Casualty Insurance ETF,Equity: U.S. Property & Casualty Insurance,Invesco,0.35%,$387.47M,0.1135,0.1179,0.1321,50.0,64.0,54.5,168.5
GRID,First Trust NASDAQ Clean Edge Smart Grid Infra...,Equity: Global Infrastructure,First Trust,0.58%,$707.91M,0.0289,0.1562,0.1321,114.0,16.0,54.5,184.5
RWJ,Invesco S&P SmallCap 600 Revenue ETF,Equity: U.S. - Small Cap,Invesco,0.39%,$1.19B,0.0220,0.1410,0.1272,124.0,24.0,68.0,216.0
RFV,Invesco S&P MidCap 400 Pure Value ETF,Equity: U.S. - Mid Cap Value,Invesco,0.35%,$305.52M,0.0877,0.1201,0.1158,63.0,58.0,123.5,244.5
PSCC,Invesco S&P SmallCap Consumer Staples ETF,Equity: U.S. Consumer Staples,Invesco,0.29%,$56.65M,0.0809,0.1062,0.1284,67.0,122.0,61.0,250.0
RGI,Invesco S&P 500 Equal Weight Industrials ETF,Equity: U.S. Industrials,Invesco,0.40%,$371.01M,0.0226,0.1163,0.1302,122.0,71.0,57.0,250.0
FIW,First Trust Water ETF,Equity: U.S. Water,First Trust,0.53%,$1.32B,0.0111,0.1365,0.1248,151.5,28.0,76.0,255.5


### Fechando navegador

In [13]:
#driver.quit()